In [214]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
pd.set_option('display.max_colwidth', None)
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.integrate import quad
import scipy.integrate as integrate
import scipy.special as special

In [220]:
xls = pd.ExcelFile('Dataset_ Exam 2.xlsx')
df = pd.read_excel(xls, 'Question_2 Estimation Dataset')
df.columns = df.columns.map(str)

In [241]:
df.head(2)

,t,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0,0.202074,0.189858,0.217837,0.208467,0.193658,0.198569,0.189084,0.209849,0.204434,...,0.209699,0.197124,0.188491,0.212023,0.191988,0.195760,0.204147,0.200000,0.194022,0.200339
1,1,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,...,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074,0.202074


## Q2 (c)

In [242]:
signals_time = {}
for index,row in df.drop(columns=['t']).iterrows():
    if len(row.dropna()):
        signals_time[index] = np.array(row.dropna())
#     print(np.array(row.dropna()))
#     print(len(row.dropna()))

In [243]:
signals_component = {}
for i in range(20):
    signals_component[i+1] =np.array(df[str(i+1)].dropna())

In [244]:
i = 1
length = len(signals_component[i])
X = np.array(range(length)).reshape((-1,1))
y = signals_component[i]
reg = LinearRegression().fit(X, y)
print(reg.score(X, y))
print(reg.coef_)
print(reg.intercept_)
print(np.sum(np.array([y[i]-reg.predict(np.array([[i]])) for i in range(length)])**2)/length)

0.8856568653238694
[0.05036692]
3.917527714380446
2.6059532052424417


In [245]:
thetas = []
sigmas = []
phis = []
for j in range(20):
    i = j+1
    length = len(signals_component[i])
    X = np.array(range(length)).reshape((-1,1))
    y = signals_component[i]
    reg = LinearRegression().fit(X, y)
    thetas.append(reg.coef_)
    phis.append(reg.intercept_)
    if j:
        sigmas.append(np.sum(np.array([y[x]-reg.predict(np.array([[x]])) for x in range(1,length)])**2)/(j*(length-1)))

In [246]:
phi = np.mean(signals_time[0])
sigma_2 = np.mean(sigmas)

In [247]:
mu0, sigma0 = norm.fit(thetas)
sigma0_2 = sigma0**2

In [248]:
phi,sigma_2,mu0,sigma0_2

(0.20115499998656788,
 0.28904353668711674,
 0.06679061723847653,
 0.00032981062685270103)

## Q2 (d)

In [249]:
df2 = pd.read_excel(xls, 'Question 2 Prediction Dataset')
df2.columns = df2.columns.map(str)

In [250]:
D2 = 21

In [251]:
df_pred = {}
for col in df2.columns[1:]:
    df_pred[col] = np.array(df2[col].dropna())

In [252]:
def mu_p(S):
    length = len(df_pred[S])
    return (sigma0_2*np.sum([(x-phi)*t for t,x in enumerate(df_pred[S])])+mu0*sigma_2)/ \
        (sigma0_2*np.sum(np.arange(length)**2)+sigma_2)
def sigma_p(S):
    length = len(df_pred[S])
    return (sigma_2*sigma0_2)/(sigma0_2*np.sum(np.arange(length)**2)+sigma_2)
def g(t,S):
    length = len(df_pred[S])
    return (mu_p(S)*(t+length-1)+phi-D2)/np.sqrt(sigma_p(S)*(t+length-1)**2+sigma_2)
def F_T(t,S):
    return (norm.cdf(g(t,S))-norm.cdf(g(0,S)))/(1-norm.cdf(g(0,S)))

In [253]:
F_T(521.068,'21')

0.49998715858905013

In [254]:
for j in range(21,31):
    i = str(j)
    def R_(t):
        return F_T(t,i)-0.5
    def R(t):
        return 1-F_T(t,i)
    soln = fsolve(R_, [10])
    print(i,soln)
    print(F_T(soln,i))
    print(integrate.quad(R, 0, 10000)[0])

21 [521.06865446]
[0.5]
521.2175429626644
22 [61.57217921]
[0.5]
61.577842695484996
23 [151.06661873]
[0.5]
151.07862142109693
24 [172.0500034]
[0.5]
172.06075892252534
25 [93.03541232]
[0.5]
93.04028516246986
26 [97.78096102]
[0.5]
97.78595503392484
27 [135.30516507]
[0.5]
135.31198320285108
28 [109.30147556]
[0.5]
109.30988407169471
29 [13.9601191]
[0.5]
14.11536259082375
30 [39.55836119]
[0.5]
39.561436050743744


## Q2 (e)

In [217]:
for j in range(21,31):
    length = len(df_pred[str(j)])
    X = np.array(range(length)).reshape((-1,1))
    y = df_pred[str(j)]
    reg = LinearRegression().fit(X, y)
    mu0 = reg.coef_
    
    i = str(j)
    phi = df_pred[i][0]
    def R_(t):
        return F_T(t,i)-0.5
    def R(t):
        return 1-F_T(t,i)
    soln = fsolve(R_, [10])
    print(i,soln)
    print(F_T(soln,i))
    print(integrate.quad(R, 0, 10000)[0])
phi = 21
mu0, sigma0 = norm.fit(thetas)

21 [524.96629743]
[0.5]
525.1180335257291
22 [61.69467394]
[0.5]
61.700355543916245
23 [150.80940623]
[0.5]
150.8213701246418
24 [171.92914714]
[0.5]
171.93988905884456
25 [92.86210952]
[0.5]
92.86697051037353
26 [97.45847458]
[0.5]
97.4634469979351
27 [135.7390788]
[0.5]
135.74593063910868
28 [108.9311664]
[0.5]
108.93952770938199
29 [13.94256444]
[0.5]
14.09847138450041
30 [39.5395837]
[0.5]
39.54265765137634


## Q3

In [256]:
df3 = pd.read_excel(xls, 'Question 3 Repair & Inventory')
df3.columns = df3.columns.map(str)

In [257]:
df3 = df3.drop(columns='t')
df3 = np.array(df3['9'])
df_pred['9'] = df3

In [258]:
df_pred.keys()

dict_keys(['21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '9'])

In [259]:
i = '9'

def R_(t):
    return F_T(t,i)-0.5
def R(t):
    return 1-F_T(t,i)
soln = fsolve(R_, [100])
print(i,soln)
print(F_T(soln,i))
print(integrate.quad(R, 0, 10000)[0])

9 [190.53847743]
[0.5]
190.55328776450833


In [208]:
def cost(days):
    

0.45856594606189316